# Análise Preliminar de Crédito

## 1. Entendimento do Problema e do Contexto

 ### O que está sendo medido?
#### O dataset mede informações financeiras e demográficas de clientes de uma instituição financeira, visando prever o risco de inadimplência.
#### A variável alvo é 'default' (0 = cliente adimplente, 1 = cliente inadimplente).


### Quais são as variáveis disponíveis?

#### - Renda, Idade, Empréstimo

## 2. Formulação de Perguntas
#### 1. Qual a proporção de clientes inadimplentes (default)?
#### 2. Existe relação entre idade e inadimplência?
#### 3. Pessoas com maior empréstimo têm maior chance de default?
#### 4. Existe correlação entre renda e empréstimo?
#### 5. Há padrões estatísticos ou outliers relevantes entre as variáveis?

## 3. Principais Observações
#### - Cerca de 14% dos clientes estão inadimplentes (default = 1).
#### - Clientes inadimplentes tendem a ter empréstimos maiores.
#### - A idade média dos inadimplentes parece ser um pouco menor.
#### - Existe correlação leve entre empréstimo e renda.
#### - Há valores negativos de idade, que foram removidos.

In [ ]:
!pip -q install plotly

In [ ]:
!pip -q install yellowbrick

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
base_credit = pd.read_csv('credit_data.csv')

In [ ]:
base_credit.rename(columns={'default': 'risk'}, inplace=True)

In [ ]:
# Mostra informações do DataFrame
base_credit

In [ ]:
#Análise Exploratória dos Dados (EDA)

# Informações iniciais
display(base_credit.info())
# Estatisticas descritivas
base_credit.describe()
display(base_credit.describe())

In [ ]:
# Correção de idades negativas
base_credit = base_credit[base_credit['age'] > 0]

In [ ]:
# Distribuição da variável alvo
graph = sns.countplot(data=base_credit, x='risk')
plt.title("Distribuição da variável de risco")
plt.xlabel("Risco (0 = Não, 1 = Sim)")
plt.ylabel("Quantidade de clientes")
plt.show()

In [ ]:
# Boxplot da idade por inadimplência
sns.boxplot(data=base_credit, x='risk', y='age')
plt.title('Idade vs Inadimplência')
plt.xlabel('Risco (0 = Não, 1 = Sim)')
plt.ylabel('Idade')
plt.show()


In [ ]:
# Valor do empréstimo por inadimplência
sns.boxplot(data=base_credit, x='risk', y='loan')
plt.title('Empréstimo vs Inadimplência')
plt.xlabel('Risco (0 = Não, 1 = Sim)')
plt.ylabel('Valor do Empréstimo')
plt.show()

In [ ]:
# Relação entre renda e empréstimo
sns.scatterplot(data=base_credit, x='income', y='loan', hue='risk')
plt.title('Renda vs Empréstimo colorido por Inadimplência')
plt.xlabel('Renda')
plt.ylabel('Empréstimo')
plt.show()

In [ ]:
# Matriz de correlação
plt.figure(figsize=(8, 6))
sns.heatmap(base_credit.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Mapa de Calor das Correlações')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


sns.countplot(data=base_credit, x='risk')
plt.title("Distribuição da variável inadimplência do cliente")
plt.xlabel("Inadimplência (0 = Não, 1 = Sim)")
plt.ylabel("Quantidade de clientes")
plt.show()


# Analisando outras variáveis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(data=base_credit, x='risk')
plt.title("Distribuição da variável de risco")
plt.xlabel("Inadimplência (0 = Não, 1 = Sim)")
plt.ylabel("Número de clientes")
plt.show()

## Feature Engineering

In [ ]:
pip install sklean

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample


In [ ]:
base_credit = pd.read_csv('credit_data.csv')
base_credit = base_credit[base_credit['age'] > 0] # remove idades negativas

In [ ]:
# Criando faixas de idade e de renda

base_credit['age_range'] = pd.cut(base_credit['age'], bins=[0,30,45,60,100], labels=['young', 'adult', 'middle age', 'elderly'])
base_credit['age_range'] = pd.cut(base_credit['income'], bins=4, labels=['low', 'low-medium', 'medium-high', 'very high'])

In [ ]:
#Balanceamento das classes (Oversampling)
#Separando as classes

df_majority = base_credit[base_credit.default == 0]
df_minority = base_credit[base_credit.default == 1]

In [ ]:
#Oversampling da minoria

df_minority_upsampled =resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)

In [ ]:
#Combinando
df_balance = pd.concat([df_majority, df_minority_upsampled])

### Modelagem Preditiva

In [ ]:
# Usando apenas variáveis numéricas para simplificar
x = df_balance[['income', 'age', 'loan']] 
y = df_balance['default'] 

In [ ]:
# Divisão treino/teste
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [ ]:
#Regressao Logistica

log_model = LogisticRegression()
log_model.fit(x_train, y_train)
y_pred_log = log_model.predict(x_test)

print("----- Regressão Logística -----") 
print(confusion_matrix(y_test,y_pred_log))
print(classification_report(y_test,y_pred_log))

### Random Forest

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_test)

print("----- Random Forest ----- ")
print(confusion_matrix(y_test,y_pred_rf))
print(classification_report(y_test,y_pred_rf))

### Visualizacoes Complementares

In [ ]:
sns.countplot(data=df_balance, x= 'age_range', hue='default')
plt.title("Inadimplência por faixa de idade")
plt.show()

In [ ]:
sns.countplot(data=df_balance, x='age_range', hue='default')
plt.title("Inadimplência por faixa de renda")
plt.show()

# Aprendizado

In [ ]:
# Filtra linhas com condição
base_credit[base_credit['age']<20]

In [ ]:
base_credit.head(10)

In [ ]:
# Acessando uma coluna com colchetes

base_credit['age']

In [ ]:
 # Agrupa e tira a média
base_credit.groupby('age').mean()

In [ ]:
base_credit.groupby('age')['income'].mean()